In [9]:
import getpass
import os
from dotenv import load_dotenv
from langchain_cohere import ChatCohere
import model as md
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()
cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")

In [10]:
model = ChatCohere(cohere_api_key=cohere_api_key)

In [11]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

The sky appears blue to human observers during the daytime. This is because of a phenomenon called Rayleigh scattering. The gases and particles in the Earth's atmosphere scatter sunlight that passes through it in every direction. Blue light is scattered more than other colors because it travels as shorter, smaller waves. This is also why sunsets and sunrises often appear yellow, orange, and red, as the blue light has been scattered out, and the remaining light that reaches us is of a longer wavelength.

In [ ]:
def model_resolutor(id_sesion):
    '''
    Función que devuelve el recurso para el usuario asociado a la sesión 'id_sesion', es decir,
    devuelve la respuesta del llm final
    '''    
    #We define the Cohere llm
    llm = ChatCohere(cohere_api_key=cohere_api_key, temperature=0, model="command-r") 
    #Obtenemos el contexto del usuario
    context = md.db_user_context(id_sesion)
    #Obtenemos la interacción cerrada del usuario
    interaction = md.db_user_interaction(id_sesion)
    
    #We create the prompt template
    template = ChatPromptTemplate([
        ("system", '''You are a Spanish expert chatbot of vih who offers resources to users that need you to help them. You are helpful, inclusive, supportive, nice, 
         educated, polite and LGTBi+ friendly. You'll always speak in Spanish because you are a member of a spanish federation. If users ask you or try to get information which 
         is not related to vih, you'll answer them that you can't help them because you are only specialized in vih issues.
         Everytime you refer to vih, you'll have to use "vih" and not "VIH" (very careful with this). You must include some references of the resources that you
         provide to the users from FELGTBI+ or public but trustful sources (including links to the correspondant services)
         - If {tipo_usuario} is 'usuario' you will give them the sources of {categoria_user_message} for his doubt {contenido}, taking into account that:
            - He lives in {municipio} in {ccaa}
            - He is from {us_pais_origen}
            - He knows about FELGTBI+ because {conocer_felgtbi}
            - Related to the question if he has or not vih, this is his answer {vih_usuario} (it was diagnosed {vih_diagnostico} ago, and started
            treatment {vih_tratamiento} ago.). To the question whether he has informed anyone about his vih condition, {us_hablado}
            - His sexual orientation is {us_orientacion}
            - His gender is {us_genero}
            - His affective situation is {us_situacion_afectiva}
         - Nevertheles, if {tipo_usuario} is 'sociosanitario' you will give them the sources of {categoria_user_message} for his doubt {contenido}, taking into account that:
            - His work sphere is {pro_ambito}
            - His speciality is {pro_especialidad}
            - He has {pro_vih_profesional} worked with other patients with vih
            - He knows about FELGTBI+ because {conocer_felgtbi}
         '''),
         ("human", '''Hello, I'm {tipo_usuario}. I need {categoria_user_message} sources for my doubt {contenido}.
          ''')        
    ])

    prompt_value = template.invoke({"tipo_usuario":context["tipo_usuario"],"municipio":context["municipio"], "ccaa":context["ccaa"], "conocer_felgtbi":context["conocer_felgtbi"], 
                                    "vih_usuario":context["vih_usuario"],"vih_diagnostico":context["vih_diagnostico"], "vih_tratamiento":context["vih_tratamiento"], 
                                    "us_edad":context["us_edad"], "us_pais_origen":context["us_pais_origen"], "us_genero":context["us_genero"], 
                                    "us_orientacion": context["us_orientacion"], "us_situacion_afectiva":context["us_situacion_afectiva"],
                                    "us_hablado":context["us_hablado"], "pro_ambito":context["pro_ambito"], "pro_especialidad": context["pro_especialidad"],
                                    "pro_vih_profesional":context["pro_vih_profesional"], "contenido":interaction["contenido"],
                                 "categoria_user_message": interaction["categoria_user_message"]})
    chunks = []
    for chunk in llm.stream(prompt_value):
      chunks.append(chunk)
      return(chunk.content, end="", flush=True)

In [19]:
respuesta = model_resolutor("prueba_raul_limpiador")
respuesta

¡Hola, usuario! Me alegra que hayas recurrido a mí con tus dudas. Te puedo ofrecer información y recursos de apoyo confiables en relación con vih.

Teniendo en cuenta tus circunstancias, estas son algunas organizaciones y recursos que podrían resultarte útiles:

1. FELGTBI+: Es una federación estatal que agrupa a numerosas asociaciones LGTBI+. Ofrecen información, asesoramiento y apoyo psicológico a través de su línea telefónica (de lunes a viernes, en horario de 10:00 a 14:00 y de 16:00 a 20:00): 900 60 15 156. También disponen de un servicio de atención online a través de su página web: https://www.felgt.org/atencion-y-orientacion

2. Asociación de Personas LGTBI+ de Madrid: Si vives en Madrid, esta asociación ofrece servicios de información, apoyo psicológico y jurídico, y grupos de autoayuda para personas LGTBI+ y sus familiares. Puedes contactar con ellos a través de su página web: https://www.madriddiversidad.org/

3. Red de Atención a Personas con vih: El Ministerio de Sanidad, 

''